In [ ]:
# import torch
# model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v3_large', pretrained=True)
# model.eval()

In [10]:
import torch
from torchvision.models import mobilenet_v3_large

# Initialize the model
model = mobilenet_v3_large()

# Load the saved state_dict into the model
model.load_state_dict(torch.load('mobilenet_v3_large.pth'))

# Set the model to evaluation mode
model.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [11]:
# Download an example image from the pytorch website
# import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
# try: urllib.URLopener().retrieve(url, filename)
# except: urllib.request.urlretrieve(url, filename)

In [12]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([-4.1927e+00, -2.0503e+00, -1.0217e+01, -1.0502e+01, -6.2262e+00,
        -4.6185e+00, -7.4292e+00,  2.2277e+00,  5.8748e+00, -4.9711e+00,
        -4.5691e+00, -3.7786e+00, -5.9400e+00, -7.6088e+00, -4.4128e+00,
        -4.8784e+00, -4.5250e+00, -4.1160e+00, -1.9274e+00, -7.5734e+00,
        -3.4837e+00, -6.2416e+00, -5.0570e+00, -4.1671e+00, -3.4238e+00,
        -6.6778e+00, -2.6290e+00, -3.4149e+00, -8.0935e+00, -3.5094e+00,
        -4.5985e+00, -3.3810e+00, -1.2962e+00, -8.1429e+00, -3.6478e+00,
        -3.1890e+00, -3.3857e+00, -5.4922e+00, -5.5387e+00, -6.4920e+00,
        -5.1329e+00, -8.7276e+00, -7.8847e+00, -8.5348e+00, -7.9882e+00,
        -8.5054e+00, -5.5887e+00, -5.0279e+00, -7.7051e+00, -5.7837e+00,
        -6.1633e+00, -7.2985e+00, -3.9971e+00, -5.0512e+00, -3.1074e+00,
        -4.5631e+00, -5.4305e+00, -6.6597e+00, -4.4328e+00, -4.1609e+00,
        -3.2838e+00, -5.5807e+00, -7.6141e+00, -5.8370e+00, -9.6701e+00,
        -7.5837e+00, -6.1189e+00, -5.4042e+00, -6.2

In [13]:
# Download ImageNet labels
!curl -O https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10472  100 10472    0     0  36839      0 --:--:-- --:--:-- --:--:-- 36873


In [14]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Samoyed 0.9735875129699707
Pomeranian 0.015272324904799461
Eskimo dog 0.002251048805192113
white wolf 0.0018997897859662771
Great Pyrenees 0.0016386007191613317
